In [1]:
import pandas as pd
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Data Preparation

In [2]:
df = pd.read_csv("../derived/final_stats_data.csv",low_memory=False)
df = df.head(5000)

In [3]:
df['delivery_type'] = df['total_run'].astype(str)
df[ 'WicketDelivery'] = df[ 'isWicketDelivery'].astype(str)

In [4]:
df.drop(['total_run' ,'isWicketDelivery','non-striker', 'extra_type',
       'non_boundary', 'player_out', 'kind',
       'fielders_involved', 'City','MatchNumber','SuperOver',
       'WonBy', 'Margin', 'method','Player_of_Match',
       'Team1Players', 'Team2Players', 'Umpire1', 'Umpire2',
       'WinningTeam', 'Team2','Date','Team1','TossWinner'], axis = 1, inplace = True)

removing some columns


In [42]:
df = df.head(1000)

# Encoding some categorical columns


In [6]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Assuming 'df' is your DataFrame
columns_to_encode = ['Season', 'BattingTeam', 'BowlingTeam', 'delivery_type', 'TossDecision', 'Venue', 'batter', 'bowler','WicketDelivery']

# Initialize the encoder
encoder = OneHotEncoder(sparse=False, handle_unknown='error')  # Use sparse=True if memory becomes an issue

# Fit and transform on the training data
one_hot_encoded = encoder.fit_transform(df[columns_to_encode])

# Get the column names for the one-hot encoded features
one_hot_column_names = encoder.get_feature_names_out(columns_to_encode)

# Create a DataFrame with the one-hot encoded features
one_hot_df = pd.DataFrame(one_hot_encoded, columns=one_hot_column_names, index=df.index)

# Drop the original categorical columns
df.drop(columns_to_encode, axis=1, inplace=True)

# Concatenate the one-hot encoded features with the original DataFrame
df_encoded = pd.concat([df, one_hot_df], axis=1)

df_encoded.columns.to_list()


/Users/prakash/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


['ID',
 'innings',
 'overs',
 'ballnumber',
 'batsman_run',
 'extras_run',
 'batter_matches_played',
 'runs_scored',
 'dismissals',
 'balls_faced',
 '0s_scored',
 '1s_scored',
 '2s_scored',
 '4s_scored',
 '6s_scored',
 'high_score',
 '25_scored',
 '50_scored',
 '75_scored',
 '100_scored',
 'strike_rate_x',
 'batting_average',
 'notout',
 'explosivity_rating',
 '0_wickets_taken',
 '1_wickets_taken',
 '2_wickets_taken',
 '3_wickets_taken',
 '4_wickets_taken',
 '5_wickets_taken',
 '6_wickets_taken',
 'bowler_matches_played',
 'runs_conceded',
 'extras_runs_conceded',
 'wickets_taken',
 'balls_bowled',
 '4s_conceded',
 '6s_conceded',
 '0s_conceded',
 '1s_conceded',
 '2s_conceded',
 'highest_conceded',
 'strike_rate_y',
 'bowling_average',
 'economy',
 'total_runs_conceded',
 'target',
 'current_score',
 'balls_left',
 'wickets_left',
 'runs_left',
 'Season_2022',
 'BattingTeam_Chennai Super Kings',
 'BattingTeam_Delhi Capitals',
 'BattingTeam_Gujarat Titans',
 'BattingTeam_Kolkata Knight R

# scaling


In [7]:
# Specify the columns to scale
columns_to_scale = ['strike_rate_x', 'batting_average', 'strike_rate_y', 'bowling_average',
                    'economy','runs_conceded','runs_scored','balls_faced','balls_bowled',
                    'batter_matches_played','0s_scored', '1s_scored', '2s_scored', '4s_scored', '6s_scored',
                     'high_score', '25_scored', '50_scored', '75_scored', '100_scored','0_wickets_taken', '1_wickets_taken',
                    '2_wickets_taken', '3_wickets_taken', '4_wickets_taken', '5_wickets_taken','6_wickets_taken',
                    'bowler_matches_played','wickets_taken','4s_conceded','6s_conceded', '0s_conceded', '1s_conceded',
                    '2s_conceded', 'highest_conceded',]

In [8]:
# Replace infinite or too large values with the median
df_encoded[columns_to_scale] = df_encoded[columns_to_scale].replace([np.inf, -np.inf], np.nan)
df_encoded[columns_to_scale] = df_encoded[columns_to_scale].fillna(df[columns_to_scale].median())

In [9]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the selected columns
df_encoded[columns_to_scale] = scaler.fit_transform(df_encoded[columns_to_scale])

In [10]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [11]:
feature_columns = df_encoded.columns.to_list()

In [12]:
len(df_encoded)

5000

In [14]:
import pandas as pd

sequence_length = 6

def create_sequences(group):
    sequences = []
    for inning in group['innings'].unique():
        inning_data = group[group['innings'] == inning]


        features = inning_data[feature_columns]

        for i in range(len(inning_data) - sequence_length + 1):
            sequence = features.iloc[i:i + sequence_length].copy()

            sequences.append(sequence)
    return sequences

# Group by the relevant columns (including one-hot encoded columns)
grouped_df = df_encoded.groupby(['ID']).apply(create_sequences)
sequences = [item for sublist in grouped_df for item in sublist]

In [15]:
sequences[0] #222521-222526 #checks only after 6 run

,ID,innings,overs,ballnumber,batsman_run,extras_run,batter_matches_played,runs_scored,dismissals,balls_faced,0s_scored,1s_scored,2s_scored,4s_scored,6s_scored,high_score,25_scored,50_scored,75_scored,100_scored,strike_rate_x,batting_average,notout,explosivity_rating,0_wickets_taken,1_wickets_taken,2_wickets_taken,3_wickets_taken,4_wickets_taken,5_wickets_taken,6_wickets_taken,bowler_matches_played,runs_conceded,extras_runs_conceded,wickets_taken,balls_bowled,4s_conceded,6s_conceded,0s_conceded,1s_conceded,2s_conceded,highest_conceded,strike_rate_y,bowling_average,economy,total_runs_conceded,target,current_score,balls_left,wickets_left,runs_left,Season_2022,BattingTeam_Chennai Super Kings,BattingTeam_Delhi Capitals,BattingTeam_Gujarat Titans,BattingTeam_Kolkata Knight Riders,BattingTeam_Lucknow Super Giants,BattingTeam_Mumbai Indians,BattingTeam_Punjab Kings,BattingTeam_Rajasthan Royals,BattingTeam_Royal Challengers Bangalore,BattingTeam_Sunrisers Hyderabad,BowlingTeam_Chennai Super Kings,BowlingTeam_Delhi Capitals,BowlingTeam_Gujarat Titans,BowlingTeam_Kolkata Knight Riders,BowlingTeam_Lucknow Super Giants,BowlingTeam_Mumbai Indians,BowlingTeam_Punjab Kings,BowlingTeam_Rajasthan Royals,BowlingTeam_Royal Challengers Bangalore,BowlingTeam_Sunrisers Hyderabad,delivery_type_0,delivery_type_1,delivery_type_2,delivery_type_3,delivery_type_4,delivery_type_5,delivery_type_6,delivery_type_7,TossDecision_bat,TossDecision_field,"Venue_Brabourne Stadium, Mumbai","Venue_Dr DY Patil Sports Academy, Mumbai","Venue_Eden Gardens, Kolkata","Venue_Maharashtra Cricket Association Stadium, Pune","Venue_Narendra Modi Stadium, Ahmedabad","Venue_Wankhede Stadium, Mumbai",batter_A Badoni,batter_A Nortje,batter_A Tomar,batter_AD Russell,batter_AK Markram,batter_AM Rahane,batter_AR Patel,batter_AT Rayudu,batter_Abhishek Sharma,batter_Arshdeep Singh,batter_Avesh Khan,batter_B Kumar,batter_D Brevis,batter_D Padikkal,batter_DA Miller,batter_DA Warner,batter_DJ Bravo,batter_DJ Hooda,batter_DP Conway,batter_DR Sams,batter_E Lewis,batter_F du Plessis,batter_GJ Maxwell,batter_HE van der Dussen,batter_HH Pandya,batter_HR Shokeen,batter_HV Patel,batter_Harpreet Brar,batter_Ishan Kishan,batter_J Suchith,batter_JC Buttler,batter_JDS Neesham,batter_JJ Bumrah,batter_JM Bairstow,batter_JM Sharma,batter_JO Holder,batter_JR Hazlewood,batter_K Kartikeya,batter_K Rabada,batter_KA Pollard,batter_KD Karthik,batter_KH Pandya,batter_KK Ahmed,batter_KL Rahul,batter_KS Bharat,batter_KS Sharma,batter_KS Williamson,batter_Kuldeep Yadav,batter_LS Livingstone,batter_Lalit Yadav,batter_M Ashwin,batter_M Jansen,batter_M Shahrukh Khan,batter_M Theekshana,batter_M Vohra,batter_MA Agarwal,batter_MJ Santner,batter_MK Lomror,batter_MM Ali,batter_MP Stoinis,batter_MR Marsh,batter_MS Dhoni,batter_MS Wade,batter_Mohammed Siraj,batter_Mohsin Khan,batter_Mukesh Choudhary,batter_N Jagadeesan,batter_N Pooran,batter_N Rana,batter_OC McCoy,batter_PBB Rajapaksa,batter_PJ Cummins,batter_PK Garg,batter_PN Mankad,batter_PP Shaw,batter_PVD Chameera,batter_PWH de Silva,batter_Q de Kock,batter_R Ashwin,batter_R Dhawan,batter_R Parag,batter_R Powell,batter_R Sanjay Yadav,batter_R Shepherd,batter_R Tewatia,batter_RA Tripathi,batter_RD Chahar,batter_RD Gaikwad,batter_RG Sharma,batter_RK Singh,batter_RM Patidar,batter_RP Meredith,batter_RR Pant,batter_RV Patel,batter_RV Uthappa,batter_Ramandeep Singh,batter_Rashid Khan,batter_S Dhawan,batter_S Dube,batter_SN Khan,batter_SN Thakur,batter_SO Hetmyer,batter_SP Jackson,batter_SP Narine,batter_SS Iyer,batter_SV Samson,batter_SW Billings,batter_Shahbaz Ahmed,batter_Shashank Singh,batter_Shubman Gill,batter_Simarjeet Singh,batter_T Stubbs,batter_TA Boult,batter_TG Southee,batter_TH David,batter_Tilak Varma,batter_UT Yadav,batter_Umran Malik,batter_V Kohli,batter_VR Iyer,batter_WP Saha,batter_Washington Sundar,batter_YBK Jaiswal,bowler_A Badoni,bowler_A Nortje,bowler_AD Russell,bowler_AR Patel,bowler_AS Joseph,bowler_Abhishek Sharma,bowler_Arshdeep Singh,bowler_Ave

In [16]:
len(sequences[0].columns)

294

check if each sequence have the same columsn


In [17]:
# Get a list of all unique columns across all sequences
all_columns = set()
for sequence in sequences:
    all_columns.update(sequence.columns)

# Ensure all sequences have the same columns
for i, sequence in enumerate(sequences):
    missing_columns = list(all_columns - set(sequence.columns))
    if missing_columns:
        # Add missing columns with NaN values
        sequences[i] = pd.concat([sequence, pd.DataFrame(columns=missing_columns)], axis=1)

In [18]:

# Iterate through each sequence
for i, sequence in enumerate(sequences):
    # Check for NaN values in the sequence
    nan_values = sequence.isna().any()
    
    # If there are any NaN values, print the column names with NaN values
    if nan_values.any():
        print(f"NaN values found in sequence {i}:")
        print(nan_values[nan_values].index.tolist())


In [19]:
# Define columns to drop
columns_to_drop = ['delivery_type_0', 'delivery_type_1', 'delivery_type_2', 'delivery_type_3',
                   'delivery_type_4', 'delivery_type_5', 'delivery_type_6', 'delivery_type_7', 
                   'WicketDelivery_0', 'WicketDelivery_1']
# Define target columns
target_columns = columns_to_drop


sequences into 3D numpy array


Extract the target column


In [27]:
import numpy as np

# Define columns to drop
columns_to_drop = ['delivery_type_0', 'delivery_type_1', 'delivery_type_2', 'delivery_type_3',
                   'delivery_type_4', 'delivery_type_5', 'delivery_type_6', 'delivery_type_7', 
                   'WicketDelivery_0', 'WicketDelivery_1']
# Define target columns
target_columns = columns_to_drop

def generate_data_batches(sequences, batch_size):
    for sequence in sequences:
        # Extract target
        target = sequence[target_columns].values[-1]

        # Drop labels and convert remaining data to numpy array
        data = sequence.drop(columns=columns_to_drop).values

        # Yield data in batches
        for i in range(0, len(data), batch_size):
            batch_data = data[i:i+batch_size]
            yield batch_data, target

# Initialize generator
batch_size = 1
data_generator = generate_data_batches(sequences, batch_size)

# Initialize lists to store concatenated data and target
data = []
targets = []

# Iterate over generator and concatenate batches directly
for batch_data, batch_target in data_generator:
    data.append(batch_data)
    targets.append(batch_target)

# Concatenate all batches into final arrays
data = np.concatenate(data, axis=0)



In [28]:
data.shape, target.shape

((28740, 284), (287400,))

In [30]:
target = target.reshape(-1, 10)
target.shape

(28740, 10)

split dataset


In [31]:
from sklearn.model_selection import train_test_split

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

# Find indices of infinite values in the training data
inf_indices_train = np.where(np.isinf(X_train))

# Get rows and columns with infinite values
inf_rows_train = inf_indices_train[0]

# Remove rows with infinite values from both feature and target arrays
X_train_cleaned = np.delete(X_train, inf_rows_train, axis=0)
y_train_cleaned = np.delete(y_train, inf_rows_train, axis=0)

# Find indices of infinite values in the testing data
inf_indices_test = np.where(np.isinf(X_test))

# Get rows and columns with infinite values
inf_rows_test = inf_indices_test[0]

# Remove rows with infinite values from both feature and target arrays
X_test_cleaned = np.delete(X_test, inf_rows_test, axis=0)
y_test_cleaned = np.delete(y_test, inf_rows_test, axis=0)


In [32]:
X_train_cleaned.shape, y_train_cleaned.shape

((22992, 284), (22992, 10))

In [33]:
X_test_cleaned.shape, y_test_cleaned.shape

((5748, 284), (5748, 10))

Dataset and dataloader


In [34]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [35]:
class CricketDataset(torch.utils.data.Dataset):
    def __init__(self, sequences, targets):
        # Convert sequences to a compatible numeric type (float32)
        self.sequences = torch.tensor(sequences, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

In [36]:
batch_size = 32
train_dataset = CricketDataset(X_train_cleaned, y_train_cleaned)
test_dataset = CricketDataset(X_test_cleaned, y_test_cleaned)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [37]:
for x, y in train_loader:
    print(x.shape, y.shape)
    break

torch.Size([32, 284]) torch.Size([32, 10])


Lstm class


In [38]:
import torch.nn.functional as F

In [39]:
class MyLSTMWithSoftmax(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyLSTMWithSoftmax, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :] # Extract the output of the last time step
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        # out = F.softmax(out, dim=1)  # Apply softmax activation
        return out

In [40]:
class MyLSTMWithWickets(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyLSTMWithWickets, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, output_size)  # Output layer for delivery type prediction
        self.fc3 = nn.Linear(32, 1)            # Output layer for wicket prediction

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # Extract the output of the last time step
        out = self.fc1(out)
        out = self.relu(out)
        delivery_type_out = self.fc2(out)  # Output for delivery type prediction
        wicket_out = self.fc3(out)         # Output for wicket prediction
        return delivery_type_out, torch.sigmoid(wicket_out)

In [41]:
input_size = data.shape[2] #96 expected
hidden_size = 64
output_size = 8

model = MyLSTMWithSoftmax(input_size, hidden_size, output_size)

IndexError: tuple index out of range

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [ ]:
model

In [ ]:
from tqdm import tqdm

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.

    # Inside the training loop
    for inputs, labels in tqdm(train_loader):

        # Assuming 'inputs' and 'labels' are torch Tensors
        outputs = model(inputs)

        # Convert labels to long data type
        labels = labels.float()

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()

        #uopdate weights
        optimizer.step()
        running_loss += loss.item()

    # Print the loss at the end of each epoch (optional)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/batch_size}')

In [ ]:
for data, label in test_loader:
    print(data.shape, label.shape)
    break

In [ ]:
torch.set_printoptions(precision=4, sci_mode=False)

In [ ]:
model.eval()

correct_predictions = 0
total_samples = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        out = model(inputs)
        y_pred = F.softmax(out, dim=1)
        # print(y_pred[0])
        # print(labels[0])
        our_predictions = torch.argmax(y_pred, dim=1)
        actual_predictions = torch.argmax(labels, dim=1)
        # print(our_predictions[0:10], actual_predictions[0:10])
        # assert(False)

        correct_predictions += torch.sum(our_predictions == actual_predictions)
        total_samples += batch_size

accuracy = correct_predictions / total_samples
print(f'Accuracy: {accuracy*100}')

In [ ]:
for data, label in test_loader:
  print(data, label)
  break

# gpu


In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

In [ ]:
model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [ ]:
num_epochs = 90

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.

    # Inside the training loop
    for inputs, labels in tqdm(train_loader):
        inputs,labels = inputs.to(device),labels.to(device)
        # Assuming 'inputs' and 'labels' are torch Tensors
        outputs = model(inputs)

        # Convert labels to long data type
        labels = labels.float()

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()

        #uopdate weights
        optimizer.step()
        running_loss += loss.item()

    # Print the loss at the end of each epoch (optional)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/batch_size}')

In [ ]:
model.eval()

correct_predictions = 0
total_samples = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs,labels = inputs.to(device),labels.to(device)
        out = model(inputs)
        y_pred = F.softmax(out, dim=1)
        # print(y_pred[0])
        # print(labels[0])
        our_predictions = torch.argmax(y_pred, dim=1)
        actual_predictions = torch.argmax(labels, dim=1)
        # print(our_predictions[0:10], actual_predictions[0:10])
        # assert(False)

        correct_predictions += torch.sum(our_predictions == actual_predictions)
        total_samples += batch_size

accuracy = correct_predictions / total_samples
print(f'Accuracy: {accuracy*100}')

# Model training done


# saving the model


In [ ]:
# Assuming 'model' is your PyTorch model
torch.save(model.state_dict(), '../model/cric_model_2nd.pth')


## model saved


# simualte match


In [ ]:
model = MyLSTMWithSoftmax(99, 64, 8)

In [ ]:
model.load_state_dict(torch.load('../model/cric_model.pth'))

In [ ]:
unique_matches = len(set(df['ID']))

In [ ]:
df.head() # is our main DataFrame

In [ ]:
unique_ids = df['ID'].unique()

id_dataframes = {}

for unique_id in unique_ids:
    id_dataframes[unique_id] = df[df['ID'] == unique_id]

In [ ]:
unique_ids

In [ ]:
id_dataframes[unique_ids[0]].iloc[:6].head()

In [ ]:

selected_match_df = id_dataframes[1312200]
selected_match_df.head(5)


In [ ]:
features = selected_match_df[['innings', 'overs', 'ballnumber', 'batsman_run', 'extras_run', 'total_run',
                               'isWicketDelivery', 'batter_matches_played', 'runs_scored', 'dismissals',
                               'balls_faced', '0s_scored', '1s_scored', '2s_scored', '4s_scored', '6s_scored',
                               'high_score', '25_scored', '50_scored', '75_scored', '100_scored', 'strike_rate_x',
                               'batting_average', 'notout', 'explosivity_rating', '0_wickets_taken', '1_wickets_taken',
                               '2_wickets_taken', '3_wickets_taken', '4_wickets_taken', '5_wickets_taken',
                               '6_wickets_taken', 'bowler_matches_played', 'runs_conceded', 'extras_runs_conceded',
                               'wickets_taken', 'balls_bowled', '4s_conceded', '6s_conceded', '0s_conceded',
                               '1s_conceded', '2s_conceded', 'highest_conceded', 'strike_rate_y', 'bowling_average',
                               'economy', 'total_runs_conceded', 'target', 'current_score', 'balls_left',
                               'wickets_left', 'runs_left', 'Season_2008', 'Season_2009', 'Season_2010', 'Season_2011',
                               'Season_2012', 'Season_2013', 'Season_2014', 'Season_2015', 'Season_2016', 'Season_2017',
                               'Season_2018', 'Season_2019', 'Season_2020', 'Season_2021', 'Season_2022',
                               'BattingTeam_Chennai Super Kings', 'BattingTeam_Deccan Chargers', 'BattingTeam_Delhi Capitals',
                               'BattingTeam_Gujarat Lions', 'BattingTeam_Gujarat Titans', 'BattingTeam_Kochi Tuskers Kerala',
                               'BattingTeam_Kolkata Knight Riders', 'BattingTeam_Lucknow Super Giants', 'BattingTeam_Mumbai Indians',
                               'BattingTeam_Pune Warriors', 'BattingTeam_Punjab Kings', 'BattingTeam_Rajasthan Royals',
                               'BattingTeam_Rising Pune Supergiant', 'BattingTeam_Royal Challengers Bangalore',
                               'BattingTeam_Sunrisers Hyderabad', 'BowlingTeam_Chennai Super Kings', 'BowlingTeam_Deccan Chargers',
                               'BowlingTeam_Delhi Capitals', 'BowlingTeam_Gujarat Lions', 'BowlingTeam_Gujarat Titans',
                               'BowlingTeam_Kochi Tuskers Kerala', 'BowlingTeam_Kolkata Knight Riders', 'BowlingTeam_Lucknow Super Giants',
                               'BowlingTeam_Mumbai Indians', 'BowlingTeam_Pune Warriors', 'BowlingTeam_Punjab Kings',
                               'BowlingTeam_Rajasthan Royals', 'BowlingTeam_Rising Pune Supergiant',
                               'BowlingTeam_Royal Challengers Bangalore', 'BowlingTeam_Sunrisers Hyderabad',
                               'TossDecision_bat', 'TossDecision_field']]

In [ ]:
features.head(5)

In [ ]:
# Convert to PyTorch tensor
features_tensor = torch.tensor(features.values, dtype=torch.float32)

In [ ]:
features_tensor = features_tensor.unsqueeze(0)

In [ ]:
features_tensor.shape

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# Create a DataLoader with batch size 1
single_match_loader = DataLoader(TensorDataset(features_tensor), batch_size=1, shuffle=False)

In [ ]:
model.eval()

In [ ]:
class_mapping = {0: 'delivery_type_0', 1: 'delivery_type_1', 2: 'delivery_type_2',
                 3: 'delivery_type_3',
                 4: 'delivery_type_4',5: 'delivery_type_5',
                 6: 'delivery_type_6',7: 'delivery_type_7'}


In [ ]:

with torch.no_grad():
    for inputs in single_match_loader:
        print(inputs)

In [ ]:
predictions = []

with torch.no_grad():
    for inputs in single_match_loader:
        # print(inputs[0].shape)
        out = model(inputs[0])
        # print(out)
        y_pred = F.softmax(out,dim=1)
        # print(y_pred[0])
        our_predictions = torch.argmax(y_pred, dim=1)
        print(our_predictions)
        predicted_class = class_mapping[our_predictions.item()]
        print(predicted_class)
        assert(False)

In [ ]:
predictions = []
previous_prediction = None

with torch.no_grad():
    for inputs in single_match_loader:
        current_input = inputs[0]  # Assuming the input tensor is in the first element of the list
        if previous_prediction is not None:
            # Update the input tensor with the previous prediction
            current_input[0, -1, -1] = previous_prediction.item()

        out = model(current_input)
        y_pred = F.softmax(out, dim=1)
        our_prediction = torch.argmax(y_pred, dim=1).item()

        # Save the prediction and update the previous prediction for the next iteration
        predictions.append(our_prediction)
        previous_prediction = torch.tensor(our_prediction, dtype=torch.float32)

# Now 'predictions' contains the predicted outcomes for each ball in the match
print(predictions)


In [ ]:
predictions = []
previous_prediction = None

# Assuming features_tensor contains the entire match data
for i in range(features_tensor.size(1)):  # Iterate through each ball in the match
    current_input = features_tensor[:, i:i+1, :]

    if previous_prediction is not None:
        # Update the input tensor with the previous prediction
        current_input[0, 0, -1] = previous_prediction.item()

    with torch.no_grad():
        out = model(current_input)
        y_pred = F.softmax(out, dim=1)
        our_prediction = torch.argmax(y_pred, dim=1).item()

    # Save the prediction and update the previous prediction for the next iteration
    predictions.append(our_prediction)
    previous_prediction = torch.tensor(our_prediction, dtype=torch.float32)

# Now 'predictions' contains the predicted outcomes for each ball in the match
print(predictions)


In [ ]:
len(predictions)

In [ ]:
predicted_selected_match_df = selected_match_df.copy()

In [ ]:
predicted_selected_match_df.head(25)

In [ ]:
predicted_selected_match_df['predicted_outcome'] = predictions

In [ ]:
column_mapping = {
    'delivery_type_0': 0,
    'delivery_type_1': 1,
    'delivery_type_2': 2,
    'delivery_type_3': 3,
    'delivery_type_4': 4,
    'delivery_type_5': 5,
    'delivery_type_6': 6,
    'delivery_type_7': 7,
}

In [ ]:
delivery_type_columns = ['delivery_type_0', 'delivery_type_1', 'delivery_type_2', 'delivery_type_3', 'delivery_type_4', 'delivery_type_5', 'delivery_type_6', 'delivery_type_7']

In [ ]:
predicted_selected_match_df['actual_outcome'] = (
    predicted_selected_match_df[delivery_type_columns].apply(lambda row: sum(row[col] * column_mapping[col] for col in delivery_type_columns), axis=1)
)

In [ ]:
predicted_selected_match_df.drop('actual_outcome',axis=1,inplace=True)

In [ ]:
predicted_selected_match_df.head(5)

In [ ]:
predicted_selected_match_df['ballnumber'].unique()

adding predicted current score


In [ ]:
predicted_selected_match_df['predicted_current_score'] = predicted_selected_match_df.groupby(['ID', 'innings'])['predicted_outcome'].cumsum()

# Reset the index if needed
predicted_selected_match_df = predicted_selected_match_df.reset_index(drop=True)

In [ ]:
predicted_selected_match_df['current_ball_number'] = predicted_selected_match_df.groupby('innings').cumcount()

In [ ]:
import matplotlib.pyplot as plt

# Sample DataFrame with predicted_outcome and actual_outcome columns
# Replace this with your actual DataFrame




# Scatter plot for predicted outcome
plt.scatter(predicted_selected_match_df['overs'], predicted_selected_match_df['ballnumber'], c=predicted_selected_match_df['predicted_outcome'], marker='o', label='Predicted Outcome')

# Scatter plot for actual outcome
plt.scatter(predicted_selected_match_df['overs'], predicted_selected_match_df['ballnumber'], c=predicted_selected_match_df['actual_outcome'], marker='x', label='Actual Outcome')

plt.xlabel('Over')
plt.ylabel('Ball Number')
plt.title('Predicted vs Actual Outcome')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Sample DataFrame with current_score, predicted_current_score, balls_left, and innings columns
# Replace this with your actual DataFrame



# Separate data for each inning
inning1_data = predicted_selected_match_df[predicted_selected_match_df['innings'] == 1]
inning2_data = predicted_selected_match_df[predicted_selected_match_df['innings'] == 2]

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 8), sharex=True)

# Plot for Inning 1
axes[0].plot(inning1_data['current_ball_number'], inning1_data['current_score'], marker='o', label='Actual Score')
axes[0].plot(inning1_data['current_ball_number'], inning1_data['predicted_current_score'], marker='x', label='Predicted Score')
axes[0].set_ylabel('Score')
axes[0].set_title('Inning 1 Actual vs Predicted Current Score')
axes[0].legend()

# Plot for Inning 2
axes[1].plot(inning2_data['current_ball_number'], inning2_data['current_score'], marker='o', label='Actual Score')
axes[1].plot(inning2_data['current_ball_number'], inning2_data['predicted_current_score'], marker='x', label='Predicted Score')
axes[1].set_xlabel('Current Ball Number')
axes[1].set_ylabel('Score')
axes[1].set_title('Inning 2 Actual vs Predicted Current Score')
axes[1].legend()

plt.tight_layout()
plt.show()
